In [4]:
import vista

world = vista.World(["a"], trace_config={'road_width': 4})
car = world.spawn_agent(config={'length': 5.,
                                'width': 2.,
                                'wheel_base': 2.78,
                                'steering_ratio': 14.7,
                                'lookahead_road': True})
display = vista.Display(world)

2023-04-27 02:07:09,537::WARNING::[d:\Research\MIT DRL\VISTA\vista\entities\sensors\EventCamera.<module>] Fail to import module for event camera. Remember to do source <some-dir>/openeb/build/utils/scripts/setup_env.shCan ignore this if not using it


FileNotFoundError: a\speed.csv not found.